# Assignment 9
## Task 2.2 – Agent with a song-lyrics tool

In this part of the assignment you will build a **simple agentic system** using the `smolagents` library.

First, make sure to read the [guided tour](https://huggingface.co/docs/smolagents/v1.23.0/en/guided_tour) of the library. It contains explanations about everything you will need to complete your tasks.

**Goal.** Your agent should be able to provide the user with *song lyrics* by:
1. Calling a custom retrieval tool `get_lyrics(title, artist=None)`, accepting a song title and optionally the artist name as inputs. The tool looks up lyrics in a Hugging Face dataset and return it as a string.
2. Using an LLM (namely gemini-2.5-flash-lite) via `smolagents` to decide when to call the tool and to return the retrieved lyrics to the user exactly as stored in the dataset.

If the artist is not provided, your tool should search by title only and return the best match.

**Example.**  
If the user asks:

> “Give me the lyrics of 'Imagine' by John Lennon.”

the agent should:
1. Call `get_lyrics(title="Imagine", artist="John Lennon")` **OR** `get_lyrics(title="Imagine", artist=None)` if no artist was provided in the user prompt  
2. Receive the raw lyrics text as tool output  
3. Read those lyrics and generate an answer like “Here are the Lyrics for the song 'Imagine' by John Lennon: Imagine there's no heaven It's easy if you try No hell below us Above us, only sky ...”


You will:
- inspect the Hugging Face lyrics dataset,
- implement the `get_lyrics` tool,
- plug it into a `ToolCallingAgent`,
- and experiment with a few queries to analyse how well the agent uses the tool.

Start by installing the necessary packages for this assigment.

In [1]:
%pip install "smolagents[openai]"; pip install datasets


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Step 1 – Explore the Hugging Face dataset


Load the [Spotify million song dataset](https://huggingface.co/datasets/vishnupriyavr/spotify-million-song-dataset) from HuggingFace and take a look at it.
Understanding which columns exist (e.g. title, artist, lyrics) will make it much easier to implement `get_lyrics`.

In [2]:
from datasets import load_dataset

dataset = load_dataset("vishnupriyavr/spotify-million-song-dataset", split="train")

# Inspect the dataset structure and a first example.
print(dataset)
print(dataset[0])
print(dataset[1])

# You can also try:
# dataset.column_names
# dataset[0].keys()

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['artist', 'song', 'link', 'text'],
    num_rows: 57650
})
{'artist': 'ABBA', 'song': "Ahe's My Kind Of Girl", 'link': '/a/abba/ahes+my+kind+of+girl_20598417.html', 'text': "Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what 

### Step 2 - Implement the custom tool

Implement the `get_lyrics` function to match the description.

In [3]:
from smolagents import tool
from typing import Optional

@tool
def get_lyrics(title: str, artist: Optional[str] = None) -> str:
    """
    Return the lyrics of the given song, exactly as stored in the dataset.

    The agent's goal is to retrieve and output the lyrics exactly
    as stored in the dataset. Matching is case-insensitive.

    Args:
        title: Song title (e.g., "Imagine").
        artist: Optional artist name (e.g., "John Lennon"). If not
            provided, the search is done only by title.

    Returns:
        The raw lyrics text exactly as stored in the dataset, or a human-
        friendly message if no match is found.
    """
    # Normalize inputs for case-insensitive comparison
    title_norm = title.strip().lower()
    artist_norm = artist.strip().lower() if artist else None

    # Try to find a matching entry in the dataset
    for entry in dataset:
        # NOTE: column names in this dataset
        entry_title = (entry.get("song") or "").strip().lower()
        entry_artist = (entry.get("artist") or "").strip().lower()
        entry_lyrics = (entry.get("text") or "")

        if artist_norm:
            if entry_title == title_norm and entry_artist == artist_norm:
                return entry_lyrics
        else:
            if entry_title == title_norm:
                return entry_lyrics

    return "Sorry, no matching song found."


Make sure that the tool works correctly before instantiating the agent and making an API call.



In [4]:
# Test AFTER you implement get_lyrics()

test_queries = [
    {"title": "Imagine"},
    {"title": "Imagine", "artist": "John Lennon"},
    {"title": "Halo"},  
    {"title": "Nonexistent Song"},
]

for q in test_queries:
    print("=" * 60)
    print("Testing:", q)
    try:
        out = get_lyrics(**q)
        print("Output (first 300 chars):")
        print(out[:300] if isinstance(out, str) else out)
    except NotImplementedError:
        print("Implement get_lyrics() first.")

Testing: {'title': 'Imagine'}
Output (first 300 chars):
Imagine there's no heaven,  
It's easy if you try,  
No hell below us,  
Above us only sky,  
Imagine all the people  
Living for today...  
  
Imagine there's no countries,  
It isn't hard to do,  
Nothing to kill or die for,  
No religion too,  
Imagine all the people  
Living life in 
Testing: {'title': 'Imagine', 'artist': 'John Lennon'}
Output (first 300 chars):
Sorry, no matching song found.
Testing: {'title': 'Halo'}
Output (first 300 chars):
Sorry, no matching song found.
Testing: {'title': 'Halo'}
Output (first 300 chars):
Good and bad  
I swear I've had them both, they're overrated  
But is it fun  
When you get hold of one  
Some gone bad  
And some gone back  
Good ones all get taken I'm callin' bluff  
You ain't strong enough  
  
Wait and pray you'll pick on me  
the day I raise my hand  
Guess that I'
Testing: {'title': 'Nonexistent Song'}
Output (first 300 chars):
Good and bad  
I swear I've had them both, they're 

### Step 3 - Load the model

To generate your API key, follow these instructions:
1. Make sure you are connected to the UZH network. Note: Eduroam is not sufficient; you need to use either UZH VPN or the uzh Wifi network.
2. Log in to the [LiteLLM gateway server](http://172.23.206.243:4000) from our course, using your UZH email and the password you set up during Assignment 1. Note: If you forgot your password, send an email to giovanni.rocci@uzh.ch, I will provide you with a password reset link.
3. Once you are logged-in, visit http://172.23.206.243:4000/ui/?page=api-keys
    - Click on the blue button "create new key"
    - Do not select any Team
    - Choose a key name
    - Under "Models", select “All Team models”
    - Create the key, copy it and paste it in the cell below under `api_key`

**Note**: do not change parameters `model_id` and `api_base` in the model definition. You are free to experiment using additional parameters related to generation (such as `temperature`, `max_tokens`, `top_p`, etc.).

In [5]:
from smolagents import OpenAIModel

with open("key.txt", "r") as f:
    api_key = f.read().strip()

model = OpenAIModel(
    model_id="gemini-2.5-flash-lite",
    api_key=api_key,
    api_base="http://172.23.206.243:4000/",
)

### Step 4 - Build and call the agent

Based on the example from the `smolagents` guided tour, provide the agent with the right arguments.

In [6]:
from smolagents import ToolCallingAgent

# TODO: insert the arguments for your agent
agent = ToolCallingAgent(tools=[get_lyrics], model=model)


#agent.run("Give me the lyrics of 'Imagine' by John Lennon.")
# agent.run("Give me the lyrics of 'Halo'.")
# agent.run("Give me the lyrics of 'Motive' by EAZ") # Not in dataset
agent.run("Give me the lyrics of 'California'") 

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me the lyrics of 'California'                                                                              │
│                                                                                                                 │
╰─ OpenAIModel - gemini-2.5-flash-lite ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_lyrics' with arguments: {'title': 'California'}                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: By Bob Dylan  
  
I'm goin' down south,  
'Neath the borderline.  
I'm goin' down south,  
'Neath the borderline.  
Some fat momma  
Kissed my mouth one time.  
  
Well, I needed it this morning  
Without a shadow of doubt.  
My suitcase is packed,  
My clothes are hangin' out.  
  
San Francisco's fine,  
You sure get lots of sun.  
San Francisco is fine.  
You sure get lots of sun.  
But I'm used to four seasons,  
California's got but one.  
  
Well, I got my dark sunglasses,  
I got for good luck my black tooth.  
I got my dark sunglasses,  
And for good luck I got my black tooth.  
Don't ask me nothin' about nothin',  
I just might tell you the truth.

[Step 1: Duration 0.68 seconds| Input tokens: 1,149 | Output tokens: 5]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "By Bob Dylan  \n  \nI'm goin' down south,  \n'Neath    │
│ the borderline.  \nI'm goin' down south,  \n'Neath the borderline.  \nSome fat momma  \nKissed my mouth one     │
│ time.  \n  \nWell, I needed it this morning  \nWithout a shadow of doubt.  \nMy suitcase is packed,  \nMy       │
│ clothes are hangin' out.  \n  \nSan Francisco's fine,  \nYou sure get lots of sun.  \nSan Francisco is fine.    │
│ \nYou sure get lots of sun.  \nBut I'm used to four seasons,  \nCalifornia's got but one.  \n  \nWell, I got my │
│ dark sunglasses,  \nI got for good luck my black tooth.  \nI got my dark sunglasses,  \nAnd for good luck I got │
│ my black tooth.  \nDon't ask me nothin' about nothin',  \nI just might tell you the truth."}                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: By Bob Dylan  
  
I'm goin' down south,  
'Neath the borderline.  
I'm goin' down south,  
'Neath the borderline.  
Some fat momma  
Kissed my mouth one time.  
  
Well, I needed it this morning  
Without a shadow of doubt.  
My suitcase is packed,  
My clothes are hangin' out.  
  
San Francisco's fine,  
You sure get lots of sun.  
San Francisco is fine.  
You sure get lots of sun.  
But I'm used to four seasons,  
California's got but one.  
  
Well, I got my dark sunglasses,  
I got for good luck my black tooth.  
I got my dark sunglasses,  
And for good luck I got my black tooth.  
Don't ask me nothin' about nothin',  
I just might tell you the truth.

Final answer: By Bob Dylan  
  
I'm goin' down south,  
'Neath the borderline.  
I'm goin' down south,  
'Neath the borderline.  
Some fat momma  
Kissed my mouth one time.  
  
Well, I needed it this morning  
Without a shadow of doubt.  
My suitcase is packed,  
My clothes are hangin' out.  
  
San Francisco's fine,  
You sure get lots of sun.  
San Francisco is fine.  
You sure get lots of sun.  
But I'm used to four seasons,  
California's got but one.  
  
Well, I got my dark sunglasses,  
I got for good luck my black tooth.  
I got my dark sunglasses,  
And for good luck I got my black tooth.  
Don't ask me nothin' about nothin',  
I just might tell you the truth.

[Step 2: Duration 1.12 seconds| Input tokens: 2,604 | Output tokens: 223]

"By Bob Dylan  \n  \nI'm goin' down south,  \n'Neath the borderline.  \nI'm goin' down south,  \n'Neath the borderline.  \nSome fat momma  \nKissed my mouth one time.  \n  \nWell, I needed it this morning  \nWithout a shadow of doubt.  \nMy suitcase is packed,  \nMy clothes are hangin' out.  \n  \nSan Francisco's fine,  \nYou sure get lots of sun.  \nSan Francisco is fine.  \nYou sure get lots of sun.  \nBut I'm used to four seasons,  \nCalifornia's got but one.  \n  \nWell, I got my dark sunglasses,  \nI got for good luck my black tooth.  \nI got my dark sunglasses,  \nAnd for good luck I got my black tooth.  \nDon't ask me nothin' about nothin',  \nI just might tell you the truth."

### Step 5 - Explore your agent's behaviour

1. **Ambiguous titles**
   - What happens when you omit the artist information for a song that shares the name with another famous song?
   - How do you think these cases should be handled?

2. **Failure modes**
   - Try a query where the song is **not** in the dataset (e.g. a very obscure title, or nonsense).  
   - How does the agent respond? 

1. Our guess is that it works thorugh the dataset in alphabetical order and it just picks the first one. TO handle these cases, it should let the user know that there are multiple songs with that title and also ask the user what information they want (e.g. all the songs, specific song from artist, random artist, etc.)


```bash
╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me the lyrics of 'California'                                                                              │
│                                                                                                                 │
╰─ OpenAIModel - gemini-2.5-flash-lite ───────────────────────────────────────────────────────────────────────────╯
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_lyrics' with arguments: {'title': 'California'}                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
Observations: By Bob Dylan  
  
I'm goin' down south,  
'Neath the borderline.  
I'm goin' down south,  
'Neath the borderline.  
Some fat momma  
Kissed my mouth one time.  
  
Well, I needed it this morning  
Without a shadow of doubt.  
My suitcase is packed,  
My clothes are hangin' out.  
  
San Francisco's fine,  
You sure get lots of sun.  
San Francisco is fine.  
You sure get lots of sun.  
But I'm used to four seasons,  
California's got but one.  
  
Well, I got my dark sunglasses,  
I got for good luck my black tooth.  
I got my dark sunglasses,  
And for good luck I got my black tooth.  
Don't ask me nothin' about nothin',  
I just might tell you the truth.
[Step 1: Duration 0.68 seconds| Input tokens: 1,149 | Output tokens: 5]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "By Bob Dylan  \n  \nI'm goin' down south,  \n'Neath    │
│ the borderline.  \nI'm goin' down south,  \n'Neath the borderline.  \nSome fat momma  \nKissed my mouth one     │
│ time.  \n  \nWell, I needed it this morning  \nWithout a shadow of doubt.  \nMy suitcase is packed,  \nMy       │
│ clothes are hangin' out.  \n  \nSan Francisco's fine,  \nYou sure get lots of sun.  \nSan Francisco is fine.    │
│ \nYou sure get lots of sun.  \nBut I'm used to four seasons,  \nCalifornia's got but one.  \n  \nWell, I got my │
│ dark sunglasses,  \nI got for good luck my black tooth.  \nI got my dark sunglasses,  \nAnd for good luck I got │
│ my black tooth.  \nDon't ask me nothin' about nothin',  \nI just might tell you the truth."}                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
Observations: By Bob Dylan  
  
I'm goin' down south,  
'Neath the borderline.  
I'm goin' down south,  
'Neath the borderline.  
Some fat momma  
Kissed my mouth one time.  
  
Well, I needed it this morning  
Without a shadow of doubt.  
My suitcase is packed,  
My clothes are hangin' out.  
  
San Francisco's fine,  
You sure get lots of sun.  
San Francisco is fine.  
You sure get lots of sun.  
But I'm used to four seasons,  
California's got but one.  
  
Well, I got my dark sunglasses,  
I got for good luck my black tooth.  
I got my dark sunglasses,  
And for good luck I got my black tooth.  
Don't ask me nothin' about nothin',  
I just might tell you the truth.
Final answer: By Bob Dylan  
  
I'm goin' down south,  
'Neath the borderline.  
I'm goin' down south,  
'Neath the borderline.  
Some fat momma  
Kissed my mouth one time.  
  
Well, I needed it this morning  
Without a shadow of doubt.  
My suitcase is packed,  
My clothes are hangin' out.  
  
San Francisco's fine,  
You sure get lots of sun.  
San Francisco is fine.  
You sure get lots of sun.  
But I'm used to four seasons,  
California's got but one.  
  
Well, I got my dark sunglasses,  
I got for good luck my black tooth.  
I got my dark sunglasses,  
And for good luck I got my black tooth.  
Don't ask me nothin' about nothin',  
I just might tell you the truth.
```

2. It does what its supposed to do and tells us that the song is not in the dataset.

``` bash
╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me the lyrics of 'Motive' by EAZ                                                                           │
│                                                                                                                 │
╰─ OpenAIModel - gemini-2.5-flash-lite ───────────────────────────────────────────────────────────────────────────╯
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_lyrics' with arguments: {'title': 'Motive', 'artist': 'EAZ'}                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
Observations: Sorry, no matching song found.
[Step 1: Duration 1.21 seconds| Input tokens: 1,153 | Output tokens: 9]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "Sorry, I couldn't find the lyrics for 'Motive' by      │
│ EAZ."}                                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
Observations: Sorry, I couldn't find the lyrics for 'Motive' by EAZ.
Final answer: Sorry, I couldn't find the lyrics for 'Motive' by EAZ.
[Step 2: Duration 1.11 seconds| Input tokens: 2,388 | Output tokens: 31]
"Sorry, I couldn't find the lyrics for 'Motive' by EAZ."
```